In [19]:
import wrds
import pandas as pd
import os
import json
from datetime import datetime
import re

In [20]:
# set the directory path
directory = "articles"

# initialize an empty list to store the data
data = []

# loop through each directory in the Articles directory
for subdir in os.listdir(directory):
    subdir_path = os.path.join(directory, subdir)
    if os.path.isdir(subdir_path):
        # loop through each file in the directory
        for filename in os.listdir(subdir_path):
            file_path = os.path.join(subdir_path, filename)
            if os.path.isfile(file_path) and filename.endswith(".json"):
                # read the JSON file
                with open(file_path, "r") as f:
                    json_obj = json.load(f)
                # extract the relevant information from the JSON object
                _id = json_obj["_id"]
                #name of the directory is the company name
                company = subdir
                title = json_obj["title"]
                text = json_obj["text"]
                cleaned_text = re.sub(r'<p>|</p>', '', text)
                #Change to timestamp YYYY-MM-DD HH:MM:SS
                published = json_obj["published"]
                timestamp = datetime.strptime(published, '%Y-%m-%dT%H:%M:%S.%fZ')
                data.append([_id, company, title, cleaned_text, timestamp])
        
df = pd.DataFrame(data, columns=["id", "company", "title", "text", "published"])

In [21]:
#Remove duplicates texts
df = df.sort_values(['company', 'published'])
df = df.drop_duplicates(subset=['company','text'], keep='first')

In [22]:
#print company names
company_to_ticker = {
    '3M_Company': 'MMM',
    'American_Express_co': 'AXP',
    'Amgen_Inc': 'AMGN',
    'Apple_Inc': 'AAPL',
    'Boeing_Co': 'BA',
    'Caterpillar_Inc': 'CAT',
    'Chevron_Corporation': 'CVX',
    'Cisco_Systems_Inc': 'CSCO',
    'Coca_Cola_Co': 'KO',
    'Dow_Inc': 'DOW',
    'Goldman_Sachs_Group_Inc': 'GS',
    'Home_Depot_Inc': 'HD',
    'Honeywell_International_Inc': 'HON',
    'Intel_Corporation': 'INTC',
    'International_Business_Machines_Corporation': 'IBM',
    'JPMorgan_Chase_Co': 'JPM',
    'Johnson_Johnson': 'JNJ',
    'McDonald_s_Corporation': 'MCD',
    'Merck_Co_Inc': 'MRK',
    'Microsoft_Corporation': 'MSFT',
    'Nike_Inc': 'NKE',
    'Procter_Gamble_Co': 'PG',
    'Salesforce_Inc': 'CRM',
    'The_Walt_Disney_Company': 'DIS',
    'Travelers_Companies_Inc': 'TRV',
    'Unitedhealth_Group_Incorporated': 'UNH',
    'Verizon_communications_Inc': 'VZ',
    'Visa_Inc': 'V',
    'Walgreens_Boots_Alliance_Inc': 'WBA',
    'Walmart_Inc': 'WMT'
}

#Replace company names with ticker symbols
df['ticker'] = df['company'].map(company_to_ticker)

In [23]:
#Helpers functions for cleaning text
def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

def remove_ftcom_and_after(text):
    index = text.find("FT.com")
    return text[:index] if index != -1 else text

def remove_source_and_after(text) : 
    index  = text.find("Source:")
    return text[:index] if index != -1 else text

def replace_PG(text):
    return text.replace("P&amp;G", "Procter & Gamble")

In [33]:
#Remove html tags
df['text'] = df['text'].apply(remove_html_tags)

#Remove FT.com and after
df['text'] = df['text'].apply(remove_ftcom_and_after)

#Remove Source: and after
df['text'] = df['text'].apply(remove_source_and_after)

#Change P&G tokens 
df['text'] = df['text'].apply(replace_PG)

#Remove &#xa0; and replace with space
df['text'] = df['text'].str.replace('&#xa0;', ' ')

#Remove "Follow @FT" and after
df['text'] = df['text'].apply(lambda x: x.split("Follow @FT")[0])

#Remove rows with "Please sign up here
df = df[df['text'] != "Please sign up here"]

In [26]:
# from difflib import SequenceMatcher

# def similar_words(text, target, similarity_threshold=0.8):
#     # Split the text into words
#     words = text.split()

#     # Find words that are similar to the target
#     similar = [word for word in words if SequenceMatcher(None, word, target).ratio() > similarity_threshold]

#     return similar

# df['similar_words'] = df.apply(lambda row: similar_words(row['text'], row['company']), axis=1)

#They are going to give us the list of words that are similar to the company name

In [27]:
ticker_to_company = {'MMM' : '3m',
                     'AXP' : 'american express',
                     'AMGN' : 'amgen',
                     'AAPL' : 'apple',
                     'BA' : 'boeing',
                     'CAT' : 'caterpillar',
                     'CVX' : 'chevron',
                     'CSCO' : 'cisco',
                     'KO' : 'coca cola',
                     'DOW' : 'dow',
                     'GS' : 'goldman sachs',
                     'HD' : 'home depot',
                     'HON' : 'honeywell',
                     'INTC' : 'intel',
                     'IBM' : 'ibm',
                     'JPM' : 'jpmorgan',
                     'JNJ' : 'johnson & johnson',
                     'MCD' : 'mcdonalds',
                     'MRK' : 'merck',
                     'MSFT' : 'microsoft',
                     'NKE' : 'nike',
                     'PG' : 'procter & gamble',
                     'CRM' : 'salesforce',
                     'DIS' : 'walt disney',
                     'TRV' : 'travelers',
                     'UNH' : 'united health',
                     'VZ' : 'verizon',
                     'V' : 'visa',
                     'WBA' : 'walgreens',
                     'WMT' : 'walmart'}

df['company_words'] = df['ticker'].map(ticker_to_company)

In [28]:
df['text'] = df['text'].str.lower()
df['sentences'] = df.apply(lambda row: [sentence for sentence in row['text'].split('. ') if row['company_words'] in sentence], axis=1)

In [31]:
#For delete the sentences that contain Listen on:
df['sentences'] = df.apply(lambda row: [sentence for sentence in row['sentences'] if 'listen on:' not in sentence], axis=1)

In [32]:
#Save the dataframe to a csv file
df.to_csv('data/news_cleaned.csv', index=False)